# Installing Postgres
There are multiple ways to install Postgres databases within Kubernetes, including Helm charts ([Bitnami HA](https://github.com/bitnami/charts/tree/master/bitnami/postgresql-ha) and [Bitnami single instance](https://github.com/bitnami/charts/tree/master/bitnami/postgresql)), standard Kubernetes manifests ([stolon](https://github.com/sorintlab/stolon/tree/master/examples/kubernetes)), Kustomize ([lodran](https://framagit.org/lordran/kustomize/postgresql)), and Kubernetes operators with Custom Resource Definitoins ([Zalando](https://github.com/zalando/postgres-operator), [CrunchyData](https://github.com/CrunchyData/postgres-operator), [StackGres](https://stackgres.io/doc/latest/install/helm/), and [KubeDB](https://kubedb.com/docs/0.12.0/setup/install/)). 

Our primary goals from our Postgres installation:
1. A functional and lightweight Postgres configuration for local development
2. A production-grade, highly available, backed-up, and replicated Postgres configuration for production
3. Functional and predictable behavior in error scenarios, including full loss of data volumes, full simultaneous loss of all HA nodes, and both unexpected and scheduled cluster termination/downtime.

The Zalando operator meets all of these criteria:
1. Simple single-instance Postgres with lightweight compute, storage, and memory requirements.
2. HA configurations with load balancing, connection pooling, rolling updates, backkups to S3, replication, and supporting 
3. Active development, with perhaps the largest number of users of any of the operators, and a welcoming and supported open-source community.

We will will install the Zalando operator and UI using Helm charts, then create our first database using the Zalando CRDs.  To start, we must first pull down the Zalando postgres-operator project using 'git':

### Install Postgres Operator

In [1]:
git clone https://github.com/zalando/postgres-operator.git

Cloning into 'postgres-operator'...
remote: Enumerating objects: 23506, done.
remote: Counting objects: 100% (625/625), done.
remote: Compressing objects: 100% (385/385), done.
remote: Total 23506 (delta 404), reused 396 (delta 216), pack-reused 22881
Receiving objects: 100% (23506/23506), 8.89 MiB | 2.51 MiB/s, done.
Resolving deltas: 100% (16816/16816), done.


In [2]:
cd postgres-operator

In [3]:
kubectl create namespace zalando

namespace/zalando created


In [4]:
helm install postgres-operator ./charts/postgres-operator -n zalando

NAME: postgres-operator
LAST DEPLOYED: Thu Feb 17 10:15:03 2022
NAMESPACE: zalando
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
To verify that postgres-operator has started, run:

  kubectl --namespace=zalando get pods -l "app.kubernetes.io/name=postgres-operator"


<div class="alert alert-block alert-warning"><b>Let this pod finish creating before proceeding. You can check with the following.</b> </div>

In [10]:
kubectl get pods -n zalando

NAME                                    READY   STATUS    RESTARTS   AGE
postgres-operator-74c8dbb8c6-cqh6h      1/1     Running   0          18m
nginx-85b98978db-jp98q                  1/1     Running   0          11m
postgres-operator-ui-54bd5d45f4-zdhl7   1/1     Running   0          8m27s


### Install Postgres Operator UI

In [5]:
helm install postgres-operator-ui ./charts/postgres-operator-ui -f ../values.postgres-operator-ui.yaml -n zalando

NAME: postgres-operator-ui
LAST DEPLOYED: Thu Feb 17 10:15:24 2022
NAMESPACE: zalando
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
To verify that postgres-operator has started, run:

  kubectl --namespace=zalando get pods -l "app.kubernetes.io/name=postgres-operator-ui"


<div class="alert alert-block alert-warning"><b>Let this pod finish creating before proceeding. You can check with the following.</b></div>

In [11]:
kubectl get pods -n zalando

NAME                                    READY   STATUS    RESTARTS   AGE
postgres-operator-74c8dbb8c6-cqh6h      1/1     Running   0          19m
nginx-85b98978db-jp98q                  1/1     Running   0          12m
postgres-operator-ui-54bd5d45f4-zdhl7   1/1     Running   0          9m3s


### Setup Test Postgres Instance via the Postgres Operator UI

Now we can visit [http://postgres-ui.k8s.example.com](http://postgres-ui.k8s.example.com) to see the GUI for the operator.  You can create a test cluster to validate that the GUI works.  Name the cluster "test" and press the green button labeled "Create cluster".


![Postgres UI Example](images/pguiexample.png)

### Setup Test Postgres Instance via the CRD

Alternatively, you can create a Custom Resource using the Zalando 'postgresqls' CRD:

In [7]:
kubectl create -f ../test-postgres.yaml

postgresql.acid.zalan.do/acid-test2 created


<div class="alert alert-block alert-warning"><b>Let this pod finish creating before proceeding. You can check with the following.</b></div>

In [8]:
kubectl get pods -n default

NAME           READY   STATUS              RESTARTS   AGE
acid-test2-0   0/1     ContainerCreating   0          3s


Now that we have verified the Postgres Operator UI and the functionality of the Custom Resource Definition, we will cleanup our PostgreSQL instances by deleting the Custom Resource instances.

### Clean up

In [10]:
kubectl delete postgresqls acid-test acid-test2 -n default

postgresql.acid.zalan.do "acid-test" deleted
postgresql.acid.zalan.do "acid-test2" deleted
